# How to Use YelpAPI:

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import json
with open('/Users/tijes/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [5]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='Baltimore, MD,21202',
                                       term='Crab Cakes')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
search_results['total']

434

It looks like there were 434 total results for this Yelp search.

In [7]:
biz = pd.DataFrame(search_results['businesses'])
biz.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidley's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/OTjVDC...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1189,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,1349.560720
1,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/9hGjo5...,False,https://www.yelp.com/biz/thames-street-oyster-...,2736,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,2090.712792


In [8]:
len(search_results['businesses'])

20

It looks like there are 20 businesses stored in the list.

# Pagination and "offset"

Pagination can be understood through this brief demonstration:

<i> Total Results vs Businesses

If we check the total# of results, we can see that we had over 400 businesses that met our search criteria.

In [9]:
## total number of matching businesses
search_results['total']

434

However, if we check our list of businesses, we will see that we only received data for 20 businesses.

In [10]:
## how many businesses in our results
len(search_results['businesses'])

20

We have over 400 results, but the length is only 20? What's going on?

If you think about the results of a google search, we usually have MANY more results that Google will display at one time.  When we scroll down to the bottom of the results, we can see that Google has divided the results into several PAGES of results, instead of one extremely long page.

This is what the Yelp Fusion is doing as well! The general term for this is "Pagination".

The Yelp API will only return a "page" of 20 results at a time.

If we want to get the next page of results, we will perform another API call, but we will add an additional argument called "offset".
- The offset is what # result to use as the FIRST result for the page.
- If we had 20 businesses in our first result, we would want to add an offset of 20. 

In [11]:
# add offset to our original api call
search_results = yelp_api.search_query(location='Baltimore, MD,21202',
                                       term='Crab Cakes',
                                       offset = 20)

Now check the results

In [12]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VLGdENTqjFJuCjg1SbW7_g,nicks-fish-house-baltimore,Nicks Fish House,https://s3-media3.fl.yelpcdn.com/bphoto/xMvMJ9...,False,https://www.yelp.com/biz/nicks-fish-house-balt...,840,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 39.26036, 'longitude': -76.61421}",[delivery],$$,"{'address1': '2600 Insulator Dr', 'address2': ...",+14103474123,(410) 347-4123,3984.496569
1,ReNdW79g_1cxrgCJpFn7cA,b-and-o-american-brasserie-baltimore,B&O American Brasserie,https://s3-media2.fl.yelpcdn.com/bphoto/STUupD...,False,https://www.yelp.com/biz/b-and-o-american-bras...,577,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 39.2899436853933, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '2 N Charles St', 'address2': '',...",+14436926172,(443) 692-6172,958.642226


You should have different results than your original call when you make the offset call.

Now you can combine the results (so far) into one dataframe using pd.concat

In [13]:
## concatenate the previous results and new results. 
businesses = pd.concat([biz, biz20],
                      ignore_index=True)
display(businesses)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidley's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/OTjVDC...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1189,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,1349.560720
1,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/9hGjo5...,False,https://www.yelp.com/biz/thames-street-oyster-...,2736,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,2090.712792
2,u65W69AhbjUlvJJBkEhGNQ,miss-shirleys-cafe-baltimore-9,Miss Shirley's Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/9FsOyV...,False,https://www.yelp.com/biz/miss-shirleys-cafe-ba...,2941,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 39.2870995, 'longitude': -76.6053...","[pickup, delivery]",$$,"{'address1': '750 E Pratt St', 'address2': '',...",+14105285373,(410) 528-5373,1028.736468
3,6am8TZAFnvND52MOz-Yctg,mamas-on-the-half-shell-baltimore,Mama's On The Half Shell,https://s3-media2.fl.yelpcdn.com/bphoto/HWY8OF...,False,https://www.yelp.com/biz/mamas-on-the-half-she...,1285,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.27986, 'longitude': -76.5752399}","[pickup, delivery]",$$,"{'address1': '2901 Odonnell St', 'address2': '...",+14102763160,(410) 276-3160,3328.825798
4,p_XPpHiZbdOccx2jFUgZJA,the-local-oyster-baltimore,The Local Oyster,https://s3-media3.fl.yelpcdn.com/bphoto/u_33ay...,False,https://www.yelp.com/biz/the-local-oyster-balt...,219,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.2958288, 'longitude': -76.6188...",[delivery],$$,"{'address1': '520 Park Ave', 'address2': None,...",+18447482537,(844) 748-2537,966.729941
5,fdo4pqfSYjQyW5_-rElfew,faidley-s-seafood-baltimore-2,Faidley’s Seafood,https://s3-media4.fl.yelpcdn.com/bphoto/7aCi2U...,False,https://www.yelp.com/biz/faidley-s-seafood-bal...,39,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 39.2915897, 'longitude': -76.6222...",[],$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107526461,(410) 752-6461,1342.513104
6,BhnCaqbJtWrC36eVxlewSw,l-p-steamers-baltimore,L P Steamers,https://s3-media3.fl.yelpcdn.com/bphoto/54nSR-...,False,https://www.yelp.com/biz/l-p-steamers-baltimor...,1680,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 39.27093, 'longitude': -76.59861}",[delivery],$$,"{'address1': '1100 E Fort Ave', 'address2': ''...",+14105769294,(410) 576-9294,2918.315725
7,1C8QSqc-erkkyAUywh501A,angies-seafood-baltimore,Angie's Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/pqkYsm...,False,https://www.yelp.com/biz/angies-seafood-baltim...,679,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.289207148922245, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '1727 E Pratt St', 'address2': No...",+14103420917,(410) 342-0917,1499.083404
8,UplfpgvI5BfuDjtolDrmsA,blackwall-hitch-baltimore,Blackwall Hitch,https://s3-media3.fl.yelpcdn.com/bphoto/YoB4qY...,False,https://www.yelp.com/biz/blackwall-hitch-balti...,529,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 39.2871515492791, 'longitude': -7...",[],$$,"{'address1': '700 E Pratt St', 'address2': Non...",+14437597176,(443) 759-7176,1013.275577
9,NQu0GgFMcAB6ucmhSNpcSQ,phillips-seafood-baltimore,Phillips Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/yZFOdQ...,False,https://www.yelp.com/biz/phillips-seafood-balt...,1506,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 39.286385925645, 'longitude': -76..."